In [1]:
import pandas as pd
import numpy as np

In [2]:
from tqdm.notebook import tqdm

In [3]:
train = pd.read_json('../data/train.json')

In [4]:
train.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000


In [5]:
# songs = pd.read_json('../data/song_meta.json')

In [6]:
# songs.head()

In [7]:
# genre = pd.read_json('../data/genre_gn_all.json', typ='series')

In [8]:
# genre = pd.DataFrame(genre).reset_index().rename(columns={'index':'gn', 0:'desc'})

In [7]:
val = pd.read_json('../data/val.json')

In [8]:
val.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000


In [10]:
train.shape, val.shape

((115071, 6), (23015, 6))

## validation data cases - no songs

In [154]:
val['num_songs'] = val['songs'].map(lambda x: len(x))

In [155]:
# 곡이 없는 경우 4370건
# 0.1898, 약 19%
val[val['num_songs']==0]

,tags,id,plylst_title,songs,like_cnt,updt_date,num_songs
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000,0
8,[스트레스],80810,리듬타면서 빡시게 운동하자!!!(스트레스 날리자):},[],127,2017-02-09 17:33:45.000,0
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000,0
17,[생각나],2380,다시 생각나는 그 사람,[],6,2016-03-19 23:53:20.000,0
20,"[피아노, 이루마, 메로디]",149069,불면증엔 아름다운 뉴에이지 곡들,[],11,2019-12-15 19:15:21.000,0
...,...,...,...,...,...,...,...
22991,[],32537,컨트리 황제 조니 캐시가 선 레코드 시절 발표한 초기 대표작,[],28,2019-06-17 14:22:48.000,0
22992,[사랑],32812,♥옷차림이 가벼워질때 부담없이 듣는음악♥,[],8,2015-05-08 00:09:01.000,0
22996,[],86721,해 저무는 밤,[],5,2016-04-27 15:32:55.000,0
23002,"[텐션, 명곡, 학창시절, 텐션업]",56570,숨어듣는명곡! 텐션올릴때듣는 그시절그노래!! (그냥명곡있을수있음주의),[],538,2020-04-18 22:02:07.000,0


# CF model
* Playlist를 사람, songs를 items로

## [추후, 지금은 제외] preprocessing - duplicated rows(subset on tags and songs)
* 중복된 플레이리스트 중, like_cnt가 높은 row만 남기기

In [12]:
# temp = train.copy()
# temp.sort_values(by='like_cnt', ascending=False, inplace=True)
# temp['tags'] = temp['tags'].astype('str')
# temp['songs'] = temp['songs'].astype('str')

# temp.drop_duplicates(subset=['tags','songs'], keep='first', inplace=True)
# nondup_index = temp.index
# train = train.iloc[nondup_index, :]

# del nondup_index, temp

# train.reset_index(drop=True, inplace=True)

## preprocessing - CF dataframe TRAIN

In [11]:
from scipy.sparse import csr_matrix

In [101]:
data, rows, cols = [], [], []

for a, b in tqdm(train[['id','songs']].values):
    for bs in b:
        rows.append(a)
        cols.append(bs)
        data.append(1)

In [102]:
train_csr = csr_matrix((data, (rows, cols)), shape=(train.id.max()+1,707989))

In [14]:
print(train_csr.count_nonzero())

5285871


In [21]:
# train_dense = train_csr.todense()

In [22]:
# print(train_dense.shape)

In [17]:
print(train.id.min(), train.id.max())

1 153428


## preprocessing - CF datafram VAL

In [18]:
val.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000


In [129]:
print(val.id.min(), val.id.max())

3 153424


In [103]:
data, rows, cols = [], [], []

for a, b in tqdm(val[['id','songs']].values):
    for bs in b:
        rows.append(a)
        cols.append(bs)
        data.append(1)

In [104]:
val_csr = csr_matrix((data, (rows, cols)), shape=(val.id.max()+1,707989))

In [24]:
# val_dense = val_csr.todense()

### intersection
* val_csr[3] 기준

In [132]:
intersection = val_csr[3].dot(train_csr.T)

In [133]:
print(intersection.shape)

(1, 153429)


In [134]:
intersection[0,0]

0

In [224]:
# for i in range(0, 153429):
#     if intersection[0, i]!=0:
#         print(i, intersection[0, i])

### union

In [128]:
# train.id.min() == 1
train_csr[1].sum()

5

In [130]:
# val.id.min() == 3
val_csr[3].sum()

66

In [123]:
# boolean type 변경
train_csr_ = train_csr.astype('bool')
val_csr_ = val_csr.astype('bool')

In [131]:
(train_csr_[1] + val_csr_[3]).sum()

71

### jaccard score

In [137]:
intersection[0,20]/((train_csr_[20]+val_csr_[3]).sum())

0.011904761904761904

In [138]:
intersection[0,141]/((train_csr_[141]+val_csr_[3]).sum())

0.024630541871921183

In [139]:
intersection[0,2638]/((train_csr_[2638]+val_csr_[3]).sum())

0.02459016393442623

### recommend

In [180]:
import itertools

In [161]:
# val_csr[3] 기준 예시
scores = {}
intersection = val_csr[3].dot(train_csr.T)

for i in tqdm(range(0, 153429)):
    if intersection[0, i]==0:
        pass
    else:
        jd = intersection[0, i]/((train_csr_[i]+val_csr_[3]).sum())
        scores[i] = jd

In [166]:
max(scores.values())

0.08411214953271028

In [167]:
min(scores.values())

0.0037735849056603774

In [169]:
scores = sorted(scores.items(), reverse=True, key=lambda x: x[1])

In [184]:
given = list(itertools.chain(*list(val[val.id==3].songs)))

In [225]:
result = []
for i in scores:
    basket = list(itertools.chain(*list(train[train.id==i[0]].songs)))
    for item in basket:
        if len(result)==100:
            break
        if item not in given:
            result.append(item)

In [226]:
len(result)

100

## recommend function

In [221]:
def recommend_100(val_id):
    scores = {} 
    intersection = val_csr[val_id].dot(train_csr.T)

    for i in tqdm(range(0, 153429)):
        if intersection[0, i]==0:
            pass
        else:
            jd = intersection[0, i]/((train_csr_[i]+val_csr_[val_id]).sum())
            scores[i] = jd
            
    scores = sorted(scores.items(), reverse=True, key=lambda x: x[1])
    
    given = list(itertools.chain(*list(val[val.id==3].songs)))
    result = []
    for i in scores:
        basket = list(itertools.chain(*list(train[train.id==i[0]].songs)))
        for item in basket:
            if len(result)==100:
                break
            if item not in given:
                result.append(item)
    return result

In [222]:
example = recommend_100(118598)

In [223]:
example

[415351,
 566502,
 412246,
 660172,
 187531,
 61612,
 151421,
 516376,
 219415,
 551157,
 654428,
 467269,
 164168,
 416023,
 680970,
 557956,
 411438,
 646988,
 157900,
 320447,
 19606,
 425357,
 219971,
 21512,
 690767,
 438915,
 149081,
 271335,
 653565,
 125214,
 328908,
 333970,
 12795,
 107413,
 162569,
 161136,
 66475,
 623047,
 367826,
 209622,
 207912,
 586389,
 292859,
 328517,
 310892,
 15774,
 381851,
 225652,
 608704,
 261743,
 268669,
 562903,
 295701,
 461170,
 78983,
 76044,
 127698,
 352137,
 243541,
 269427,
 455806,
 703323,
 638488,
 422438,
 592526,
 413459,
 545816,
 693580,
 359986,
 611201,
 520034,
 412305,
 110696,
 350062,
 472374,
 104455,
 520437,
 373313,
 151080,
 656598,
 288665,
 694423,
 308797,
 637900,
 262430,
 551161,
 434094,
 149051,
 607850,
 101286,
 620311,
 584108,
 696876,
 165237,
 525935,
 491808,
 5418,
 340089,
 651465,
 303904]

### 결과 확인

In [228]:
songs = pd.read_json('../data/song_meta.json')
songs.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4


In [229]:
# seed
for i in given:
    print(songs[songs.id==i].song_name)

174104    Lovesick
Name: song_name, dtype: object
377243    돌아오지마 (Feat. 용준형 Of 비스트)
Name: song_name, dtype: object
442123    1cm의 자존심
Name: song_name, dtype: object
590253    Someone Who Can Dance
Name: song_name, dtype: object
372171    City
Name: song_name, dtype: object
142813    밖에 비온다 주룩주룩 (Rain Shower G-Mix)
Name: song_name, dtype: object
31529    Samsara (Feat. Emila)
Name: song_name, dtype: object
62003    희재
Name: song_name, dtype: object
435187    봄날은 간다
Name: song_name, dtype: object
2454    잡쉈잖아
Name: song_name, dtype: object
346913    The Lady (Feat. Bumkey & Dok2)
Name: song_name, dtype: object
368237    Too Late (Feat. Bumkey, Paloalto)
Name: song_name, dtype: object
654019    집앞에서 (전화 받지 마 Pt. 2) (Feat. Crucial Star)
Name: song_name, dtype: object
526161    Talk Dirty (Feat. 2 Chainz)
Name: song_name, dtype: object
326500    첫눈 오던 밤 (Feat. 팔로알토)
Name: song_name, dtype: object
559102    Up All Night (Feat. Mayson The Soul)
Name: song_name, dtype: object
122384    100 (F

In [231]:
# recommendation
for i in result:
    print(songs[songs.id==i].song_name)

576231    안갯길 (Prod. By 진영(B1A4))
Name: song_name, dtype: object
698865    다정하게, 안녕히
Name: song_name, dtype: object
351342    끝
Name: song_name, dtype: object
73547    Dancing King
Name: song_name, dtype: object
376358    사랑해요
Name: song_name, dtype: object
675500    사랑이 아프다
Name: song_name, dtype: object
183579    휘파람
Name: song_name, dtype: object
678936    태풍 (The Eye)
Name: song_name, dtype: object
280830    내 마음 들리나요 (Feat. 이하이)
Name: song_name, dtype: object
473756    결정 (Feat. 아이유)
Name: song_name, dtype: object
462446    그게 뭐라고
Name: song_name, dtype: object
292825    그대라는 사치
Name: song_name, dtype: object
688785    Say Yes
Name: song_name, dtype: object
64052    우주를 줄게
Name: song_name, dtype: object
216696    나만 안되는 연애
Name: song_name, dtype: object
375450    구르미 그린 달빛
Name: song_name, dtype: object
228031    그대를 잊는다는 건
Name: song_name, dtype: object
99948    꽐라
Name: song_name, dtype: object
133190    러시안 룰렛 (Russian Roulette)
Name: song_name, dtype: object
244808    몸
Name: 